# Experiment 030

In this notebook, we validate the GameGAN generator architecture on the Tetris problem by training the convolutional encoder and the rendering engine as an autoencoder (excluding the dynamics engine, events and random inputs). This should show whether the GameGAN generator architecture is suitable for the Tetris problem and the current dataset.

In [1]:
import os
from pathlib import Path
import shutil
import datetime

import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.utils.data import Dataset
import numpy as np
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt


from models import TetrisModel, TetrisDiscriminator
import metrics
from recording import FileBasedDatabaseWithEvents
from engines import EVENT_NAMES

In [2]:
NUM_CELL_TYPES = 8
NUM_EVENT_TYPES = 5

class RecordingDataset(Dataset):
    def __init__(self, path: str):
        self._db = FileBasedDatabaseWithEvents(path)

    def __len__(self):
        return len(self._db)

    def __getitem__(self, idx):
        boards, events = self._db[idx]
        b = self._transform_board(boards[0]) # Just take one board
        return b
    
    def _transform_board(self, board):
        board = torch.tensor(board, dtype=torch.long)
        board = F.one_hot(board, NUM_CELL_TYPES) # One-hot encode the cell types
        board = board.type(torch.float) # Convert to floating-point
        board = board.permute((2, 0, 1)) # Move channels/classes to dimension 0
        return board

In [3]:
# Put datasets in memory for faster training
train_dataset = list(RecordingDataset(os.path.join("data", "tetris_emulator", "train")))
test_dataset = list(RecordingDataset(os.path.join("data", "tetris_emulator", "test")))
batch_size = 12 # Was 4
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

b = next(iter(train_dataloader))
print(f"b: shape {b.shape}, dtype {b.dtype}")

b: shape torch.Size([12, 8, 22, 10]), dtype torch.float32


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using device {device}")

Using device cuda


In [5]:
real_label = 1.0
fake_label = 0.0

def train_loop(dataloader, gen, loss_fn, optimizer_gen):
    gen.train()

    size = len(dataloader.dataset)
    for batch, b in enumerate(dataloader):
        b = b.to(device)

        gen.zero_grad()
        # Reconstruct some examples
        b_pred = gen(b)
        # Calculate the generator's loss based on this output
        err_gen = loss_fn(b_pred, b)
        # Calculate gradients for generator
        err_gen.backward()
        # Update generator
        optimizer_gen.step()

        # Output training stats
        if batch % 30 == 0:
            current = batch * dataloader.batch_size + batch_size
            print(f"[{current}/{size}] G loss: {err_gen.item():.4f}")


def test_loop(split_name, dataloader, gen, loss_fn, tb_writer, epoch):
    gen.eval()

    loss_gen = 0.0
    cell_accuracy = metrics.CellAccuracy()
    board_accuracy = metrics.BoardAccuracy()

    num_batches = len(dataloader)
    with torch.no_grad():        
        for batch, b in enumerate(dataloader):
            b = b.to(device)

            b_pred = gen(b)
            
            loss_gen += loss_fn(b_pred, b).item()

            classes_b = torch.argmax(b, dim=1)
            classes_b_pred = torch.argmax(b_pred, dim=1)
            cell_accuracy.update_state(classes_b_pred, classes_b)
            board_accuracy.update_state(classes_b_pred, classes_b)


    loss_gen /= len(dataloader.dataset)

    print(f"{split_name} error: \n G loss: {loss_gen:>8f}, cell accuracy: {(cell_accuracy.result()):>0.1%}, board accuracy: {(board_accuracy.result()):>0.1%} \n")

    tb_writer.add_scalar(f"Loss/{split_name}", loss_gen, epoch)
    tb_writer.add_scalar(f"Cell accuracy/{split_name}", cell_accuracy.result(), epoch)
    tb_writer.add_scalar(f"Board accuracy/{split_name}", board_accuracy.result(), epoch)

In [6]:
def train(gen_factory, epochs, learning_rate):

    gen = gen_factory()

    loss_fn = nn.MSELoss() # BCE loss causes instability when the loss gets too small
    optimizer_gen = torch.optim.Adam(gen.parameters(), lr=learning_rate)

    log_dir = os.path.join("runs", "experiment_030")
    log_subdir = os.path.join(log_dir, datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
    tb_writer = SummaryWriter(log_subdir)

    try:
        for epoch in range(epochs):
            print(f"Epoch {epoch}\n-------------------------------")
            train_loop(train_dataloader, gen, loss_fn, optimizer_gen)
            test_loop("train", train_dataloader, gen, loss_fn, tb_writer, epoch)
            test_loop("test", test_dataloader, gen, loss_fn, tb_writer, epoch)
            gen_zero_grads = 0
            for name, weight in gen.named_parameters():
                tb_writer.add_histogram(f"Weights/{name}", weight, epoch)
                if weight.grad is not None:
                    tb_writer.add_histogram(f"Gradients/{name}", weight.grad, epoch)
                    gen_zero_grads += weight.grad.numel() - weight.grad.count_nonzero().item()
            tb_writer.add_scalar(f"Zero gradients", gen_zero_grads, epoch)
    finally:
        tb_writer.close()
        
    print("Done!")
    return gen

In [7]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [8]:
NUM_RANDOM_INPUTS = 4


class Conv2dLeakyReLU(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, use_batch_norm=False, negative_slope=0.0):
        super().__init__()

        self.use_batch_norm = use_batch_norm

        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=(not use_batch_norm))
        nn.init.kaiming_uniform_(self.conv.weight, a=negative_slope)
        if not use_batch_norm:
            nn.init.constant_(self.conv.bias, 0.01)

        if use_batch_norm:
            self.norm = nn.BatchNorm2d(out_channels)

        self.relu = nn.LeakyReLU(negative_slope)
    
    def forward(self, x):
        x = self.conv(x)
        if self.use_batch_norm:
            x = self.norm(x)
        x = self.relu(x)
        return x


class ConvTranspose2dLeakyReLU(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, output_padding=0, use_batch_norm=False, negative_slope=0.0):
        super().__init__()

        self.use_batch_norm = use_batch_norm

        self.conv = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, output_padding=output_padding, bias=(not use_batch_norm))
        nn.init.kaiming_uniform_(self.conv.weight, a=negative_slope)
        if not use_batch_norm:
            nn.init.constant_(self.conv.bias, 0.01)

        if use_batch_norm:
            self.norm = nn.BatchNorm2d(out_channels)
            
        self.relu = nn.LeakyReLU(negative_slope)
    
    def forward(self, x):
        x = self.conv(x)
        if self.use_batch_norm:
            x = self.norm(x)
        x = self.relu(x)
        return x


class LinearLeakyReLU(nn.Module):
    def __init__(self, in_features, out_features, negative_slope=0.0):
        super().__init__()

        self.linear = nn.Linear(in_features, out_features)
        nn.init.kaiming_uniform_(self.linear.weight, a=negative_slope)
        nn.init.constant_(self.linear.bias, 0.01)

        self.relu = nn.LeakyReLU(negative_slope)

    def forward(self, x):
        x = self.linear(x)
        x = self.relu(x)
        return x


class GameganGenerator(nn.Module):
    def __init__(self, use_batch_norm=False, leak=0.0):
        super().__init__()

        self.use_batch_norm = use_batch_norm
        self.leak = leak

        self.board_encoder = nn.Sequential(
            Conv2dLeakyReLU(NUM_CELL_TYPES, 64, kernel_size=2, stride=2, use_batch_norm=use_batch_norm, negative_slope=leak),
            Conv2dLeakyReLU(64, 64, kernel_size=3, use_batch_norm=use_batch_norm, negative_slope=leak),
            Conv2dLeakyReLU(64, 64, kernel_size=3, use_batch_norm=use_batch_norm, negative_slope=leak),
            nn.Flatten(start_dim=1),
            LinearLeakyReLU(448, 256, negative_slope=leak)
        )

        self.renderer = nn.Sequential(
            LinearLeakyReLU(256, 448, negative_slope=leak),
            nn.Unflatten(dim=1, unflattened_size=(64, 7, 1)),
            ConvTranspose2dLeakyReLU(64, 64, kernel_size=3, use_batch_norm=use_batch_norm, negative_slope=leak),
            ConvTranspose2dLeakyReLU(64, 64, kernel_size=3, use_batch_norm=use_batch_norm, negative_slope=leak),
            nn.ConvTranspose2d(64, NUM_CELL_TYPES, kernel_size=2, stride=2),
            nn.Softmax(dim=1),
        )

    def forward(self, b):
        # Encode board state
        s = self.board_encoder(b)

        # Render new board
        y = self.renderer(s)
        return y


print(f"Parameters: {count_parameters(GameganGenerator())}")

Parameters: 381960


In [10]:
del gen

gen = train(
    gen_factory=lambda: GameganGenerator(use_batch_norm=True, leak=0.2).to(device),
    epochs=1000,
    learning_rate=1e-4
)

Epoch 0
-------------------------------
[12/2000] G loss: 0.1178
[372/2000] G loss: 0.1086
[732/2000] G loss: 0.1002
[1092/2000] G loss: 0.0943
[1452/2000] G loss: 0.0814
[1812/2000] G loss: 0.0747
train error: 
 G loss: 0.006062, cell accuracy: 60.3%, board accuracy: 0.0% 

test error: 
 G loss: 0.006105, cell accuracy: 60.1%, board accuracy: 0.0% 

Epoch 1
-------------------------------
[12/2000] G loss: 0.0733
[372/2000] G loss: 0.0715
[732/2000] G loss: 0.0647
[1092/2000] G loss: 0.0595
[1452/2000] G loss: 0.0534
[1812/2000] G loss: 0.0522
train error: 
 G loss: 0.004227, cell accuracy: 72.2%, board accuracy: 0.0% 

test error: 
 G loss: 0.004274, cell accuracy: 71.9%, board accuracy: 0.0% 

Epoch 2
-------------------------------
[12/2000] G loss: 0.0504
[372/2000] G loss: 0.0475
[732/2000] G loss: 0.0436
[1092/2000] G loss: 0.0428
[1452/2000] G loss: 0.0438
[1812/2000] G loss: 0.0352
train error: 
 G loss: 0.003000, cell accuracy: 81.0%, board accuracy: 0.0% 

test error: 
 G lo

Having run a number of training sessions, I found the following:
* MSE loss is better than BCE for the autoencoder. BCE loss becomes very unstable when the loss gets too low and shoots back up, close to the original value. The normal advice is to use a `LogSigmoid` layer and `BCEFromLogitsLoss`, but we won't be able to do that when the encoder is part of a the full GameGAN generator.
* A shallower convolutional network with kernel size 3 is better than a deeper one with kernel size 2. The latter took many more epochs to learn, and still did not quite reach the same board accuracy.
* Increasing the channels in the hidden convolutional layers from 32 to 64 helped a lot.
* Keeping batch norm in is better than removing it. Without batch norm, the model learns a lot more slowly.
* Changing the "leak" (Leaky ReLU negative slope) from 0.2 to 0.1 makes the model learn more slowly.
* Reducing the stride in the x-direction to make the intermediate boards more square and increase the number of spatial features at the `Flatten` layer increases the number of model parameters from 380K to over 1M, but doesn't improve performance, even after trying out variations of this.
* Learning rate 1e-5 has no significant advantage over 1e-4. With 1e-5, the loss and accuracy curves are smoother, but the final values attained are not quite as good.
* Changing the batch size from 4 to 12 sped up the training (with in-memory data) from 50 mins to 20 mins, improved training board accuracy, and test board accuracy stayed comparable.

The final board accuracy of the last model was 99.85% on the training set and 81.00% on the test set.

# Conclusion

I've found an autoencoder architecture that works quite well with about 380K parameters. The model overfits significantly, so to get the best performance, we need more data.